In [1]:
# 导入tensorflow
import tensorflow as tf

In [4]:
from tensorflow.examples.tutorials.mnist import input_data

tensorflow的一个简单使用，步骤设计公式，使用eval或者run喂入数据，并执行设计好的操作  
本例子的过程：  
1、定义目标公式，即预测值的计算  
2、选择loss方法，确定优化目标，如最小化loss  
3、训练模型，迭代的喂入数据集合，如一个一个batch  
4、使用测试数据对模型进行评测  

# 载入mnist数据集

In [7]:
# 读取数据
mnist = input_data.read_data_sets('MNIST_data/',one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [8]:
# 训练数据
print(mnist.train.images.shape),print(mnist.train.labels.shape)

(55000, 784)
(55000, 10)


(None, None)

In [9]:
# 测试数据
print(mnist.test.images.shape),print(mnist.test.labels.shape)

(10000, 784)
(10000, 10)


(None, None)

In [10]:
# 验证数据
print(mnist.validation.images.shape),print(mnist.validation.labels.shape)

(5000, 784)
(5000, 10)


(None, None)

In [29]:
# 载入tensorflow后，需要注册session，session是相互独立的，会运行session中的操作。
# InteractiveSession是注册是一个session，所有操作默认使用这个session
sess = tf.InteractiveSession()

# 定义目标函数

In [30]:
# 使用placeholder，定义输入的数据。该函数第一个参数是数据类型，第二个参数是数据的维度，如[None, 784]，None表示行数不限制，列统一是784列。
x = tf.placeholder(tf.float32,[None, 784])

In [31]:
# 定义变量，在模型训练过程中，会不断更新。变量需要进行初始化，在复杂网络中，初始化方法很重要。
# 变量与placeholder的于别是，placeholder使用掉，就会消失。
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))

In [32]:
# 简单的softmax方法，y=softmax(Wx+b)，转化为tensorflow的代码如下
y = tf.nn.softmax(tf.matmul(x,W)+b)

In [28]:
# 使用tensorflow的一大有点是，他提供了自动求导，使用梯度下降进行forward和backward
# 现在有了sorftmax预测目标的方法，只需定义一个损失函数（loss）就可以确定目标函数了，用梯度下降最小化目标函数即可
# loss越小，说明预测值与真实值的  偏差  越小

交叉熵是一种，可以作为损失函数，用于度量模型对真实概率分布估计的准确度

$ H_{y'}(y)=-\sum_{i}y_{i}' log(y_i) $

其中$y'$是真实的概率分布（labels的onehot编码），y是预测的概率分布

In [33]:
# 用tesorflow语句自己定义交叉熵
# y_预测为真实的概率分布
y_ = tf.placeholder(tf.float32,[None, 10])
# tf.reduce_mean对每个batch结果求均值，tf.reduce_sum是实现公式中的求和，reduction_indices求和的维度，现在是axis了
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

In [34]:
# GradientDescentOptimizer可以直接对目标函数引用梯度下降和反向传播进行求解，初学者会用就行。
# 如果想要设计新的优化算法，或者对比不同的优化算法，那需要了解求解过程
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

# 初始化变量并训练模型

In [40]:
# 模型设置好后，训练模型前，需要对所有变量进行初始化
init = tf.global_variables_initializer().run()

In [42]:
# 训练
for i in range(10000):
    # 调用方法获取一个batch的数据，100个样本
    batch_xs, batch_ys = mnist.train.next_batch(100)
    # 用run运行设计好的模型，run中需要给之前定义的placeholder对象喂数据
    train_step.run({x: batch_xs, y_:batch_ys})

# 计算准确度

In [43]:
# 得到模型后，可以使用模型进行预测类标和真实类标进行比较
# 这里用到一个函数 tf.argmax，将概率最大的位置作为类标
# tf.equal将预测类标与真实类标作==运算,他返回的是bool类型的tensor
correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(y_,1))

In [44]:
# 得到预测正确与否的判断后，我们对bool型的tensor做类型转换，转换为tf.float32，再使用tf.reduce_mean求出1的比列，也就是准确率
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [45]:
# 定义好计算accuracy的计算过程后，可以使用内部的eval方法输入数据并计算结果
print(accuracy.eval({x:mnist.test.images, y_:mnist.test.labels}))

0.9221
